In [1]:
import threading
import time
import logging
import sys
from io import StringIO
from fastmcp import FastMCP, Client
from fastmcp.client.transports import StdioTransport

# Set root logger to CRITICAL
logging.getLogger().setLevel(logging.CRITICAL)

# Suppress all MCP/uvicorn/FastAPI related loggers
loggers_to_suppress = [
    "uvicorn",
    "uvicorn.access",
    "uvicorn.error",
    "fastmcp"
]

for logger_name in loggers_to_suppress:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.CRITICAL)
    logger.propagate = False

# Also disable the default stream handler
logging.disable(logging.WARNING)

In [2]:
# Create main server
mcp = FastMCP("Movies Server")

# Create stdio transport for Neo4j MCP server
transport = StdioTransport(
    command="uvx",
    args=["mcp-neo4j-cypher@0.5.1", "--namespace", "movies"],
    env={
        "NEO4J_URI": "neo4j+s://demo.neo4jlabs.com",
        "NEO4J_USERNAME": "recommendations",
        "NEO4J_PASSWORD": "recommendations",
        "NEO4J_DATABASE": "recommendations"
    }
)

# Create client with the transport
backend = Client(transport)

# Create and mount proxy (not async)
proxy = FastMCP.as_proxy(backend, name="Neo4jProxy")
mcp.mount(proxy, prefix="movies")

# Run server silently in background thread
def run_server():
    mcp.run(transport="streamable-http", port=8000)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)

print("✓ Server ready with Neo4j tools mounted")
print("Server running at: http://localhost:8000")

╭──────────────────────────────────────────────────────────────────────────────╮                  
                 │                                                                              │                  
                 │                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │                  
                 │                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │                  
                 │                                                                              │                  
                 │                                FastMCP 2.13.1                                │                  
                 │                                                                              │                  
                 │                                                                              │                  
                 │                  🖥  Server name: Movies Server                               │                  
                 │                                                                              │                  
                 │                  📦 Transport:   HTTP                                        │                  
                 │                  🔗 Server URL:  http://127.0.0.1:8000/mcp                   │                  
                 │                                                                              │                  
                 │                  📚 Docs:        https://gofastmcp.com                       │                  
                 │                  🚀 Hosting:     https://fastmcp.cloud                       │                  
                 │                                                                              │                  
                 ╰──────────────────────────────────────────────────────────────────────────────╯

✓ Server ready with Neo4j tools mounted
Server running at: http://localhost:8000


In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

async def query(q: str):
    client = MultiServerMCPClient({
        "neo4j-database": {
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http"
        }
    })
    
    tools = await client.get_tools()
    agent = create_react_agent(ChatOpenAI(model="gpt-5-mini"), tools)
    response = await agent.ainvoke({"messages": [{"role": "system", "content": """You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies. Never ask follow-up questions regarding the style of response. Be concise"""},
                                                 {"role": "user", "content": q}]})
    for message in response["messages"]:
        message.pretty_print()

In [4]:
await query("Which actor has played in the most movies after 2010?")

================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies. Never ask follow-up questions regarding the style of response. Be concise
================================ Human Message =================================

Which actor has played in the most movies after 2010?
================================== Ai Message ==================================
Tool Calls:
  movies_movies-read_neo4j_cypher (call_GAwICDOqmjWaT57hfPljfCfi)
 Call ID: call_GAwICDOqmjWaT57hfPljfCfi
  Args:
    query: MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE coalesce(m.released,0) > 2010
WITH p, count(distinct m) AS movieCount
WITH max(movieCount) AS maxCount
MATCH (p2:Person)-[:ACTED_IN]->(m2:Movie)
WHERE coalesce(m2.released,0) > 2010
WITH p2, count(distinct m2) AS c, maxCount
WHERE c = maxCount
RETURN p2.name AS actor, c A

In [5]:
await query("Find movies where both aliens and kids appear?")

================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies. Never ask follow-up questions regarding the style of response. Be concise
================================ Human Message =================================

Find movies where both aliens and kids appear?
================================== Ai Message ==================================
Tool Calls:
  movies_movies-get_neo4j_schema (call_DOdbPNCi4EhfhXAVki3mGnI4)
 Call ID: call_DOdbPNCi4EhfhXAVki3mGnI4
  Args:
    sample_size: 1000
================================= Tool Message =================================
Name: movies_movies-get_neo4j_schema

{"User": {"type": "node", "count": 671, "properties": {"name": {"indexed": true, "type": "STRING"}, "userId": {"indexed": true, "type": "STRING"}}, "relationships": {"RATED": {"direction": "out", 

In [7]:
await query("Create a profitability report for movies per genre per decade.")

================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies. Never ask follow-up questions regarding the style of response. Be concise
================================ Human Message =================================

Create a profitability report for movies per genre per decade.
================================== Ai Message ==================================
Tool Calls:
  movies_movies-get_neo4j_schema (call_kPTBGj1bz9ot9wNR6CN1pGJ8)
 Call ID: call_kPTBGj1bz9ot9wNR6CN1pGJ8
  Args:
    sample_size: 1000
================================= Tool Message =================================
Name: movies_movies-get_neo4j_schema

{"User": {"type": "node", "count": 671, "properties": {"name": {"indexed": true, "type": "STRING"}, "userId": {"indexed": true, "type": "STRING"}}, "relationships": {"RATED": {"dir